## 5-3. 나이브 베이즈 분류기를 이용한 문서 분류
대싱 믄서로부터 특성을 추출함으로써 머신러닝을 적용할 준비가 됐다. 먼저 문서 분류에서 가장 많이 사용돼왔고, 또 의외로 좋은 성능을 보이는 나이브 베이즈(Naive Bayes) 분류기를 적용해본다. 

5개의 기사에 대해 특성을 추출한 결과가 아래와 같이 추출됐다고 할 때, 이 중에서 세 개는 경제 관련 기사이고 두 개는 과학 관련 기사이다.
- 경제 : 인플레이션, 인플레이션, 브릭스, 4차산업혁명
- 경제 : 물가지수, 물가지수, GDP, 인플레이션, 브릭스
- 과학 : 4차산업혁명, 인공지능, 인공지능, 머신러닝
- 과학 : 인공지능, 우주산업, 우주산업, 딥러닝, 딥러닝
- 경제 : 4차산업혁명, 4차산업혁명, 우주산업, GDP

만일 특성에 대한 정보가 전혀 없다면, 새로운 기사가 있을 때 그 기사가 경제 기사일 확룰과 과학 기사일 확률을 어떻게 계산할 수 있을까? 경제 기사는 5개 중 3개이므로 3/5, 과학 기사는 2/5로 봄이 타당할 것이다. 이것을 **'사전확률'**이라고 한다. 그러나 우리에게는 특성에 대한 정보가 있으므로 이보다는 더 잘 예측할 수 있다.

특성 정보는 어떻게 활용할 수 있을까? 아주 직관적으로 봤을 때, 우리는 학습 데이터셋을 통해 어떤 단어가 경제 기사에서 더 많이 나오는지 알 수 있다. 예를 들어 위에서 인플레이션이라는 단어는 과학 기사에는 등장하지 않고 경제 기사에만 등장하므로, 인플레이션이 새로운 기사에 있다면 그 기사를 경제 기사라고 봄이 타당하다. 즉, 새로운 기사에 나온 단어들이 각각 어떤 확률로 경제관련 기사와 과학 기사에 나오는지를 계산하고 이를 잘 결합할 수만 있다면 새로운 기사가 어느 분류에 속할지 알 수 있을 것이다.

통계를 제대로 공부한 적 없는 사람에게 나이브 베이즈의 식은 너무나 어렵게 보일 수 있다. 그러나 아무리 어려운 식도 그 식을 고안한 사람의 아이디어는 단순한 사실에서 출발한 경우가 많다. 나이브 베이즈의 의도를 직관적으로 이해할 수 있다면, 각 분류에 속한 기사들의 수를 이용해 사전확률을 계산하고, 특성을 고려해 이를 더 정확한 확률로 바꾸는 과정을 이해하는 것도 그리 어려운 일은 아니나, 이 책에서는 직관적인 이해에 만족하기로 한다.

사이킷런은 sklearn.naive_bayes에서 나이브 베이즈를 위한 클래스들을 제공한다. 여기서는 그중에서 [MultinomialNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)를 이용해 20 뉴스그룹 기사들을 분류해보기로 한다. MultinomialNB는 이산적인 특성 값들을 이용해 분류를 하고자 할 때 사용한다. '이산적'이란 '연속적인 값이 아닌 값'이라는 뜻으로 우리의 경우 카운트 벡터가 이에 해당한다. 사이킷런이 제공하는 다른 나이브 베이즈 클래스로 BernoulliNB가 있는데, 이 모델은 특성 값이 이진 값일때 주로 사용한다. 원칙적으로 MultinomialNB는 이산적인 값에 대해 사용하도록 되어 있지만, TF-IDF와 같은 연속적인 값에 대해서도 잘 작동한다. 우리 입장에서는 성능이 가장 중요하므로 둘 다 해보고 더 나은 모형을 선택하면 된다. 아래 예시에서는 MultinomialNB를 불러와서 객체를 생성하고 학습한 후 학습 데이터셋과 평가 데이터셋에 대해 정확도를 계산했다. 앞에서 카운트 벡터로 특성을 추출했으니, 그대로 가져다 써본다.

In [1]:
% store -r X_train
% store -r y_train
% store -r X_test
% store -r y_test
% store -r X_train_cv
% store -r X_test_cv
% store -r newsgroups_train
% store -r newsgroups_test

In [2]:
from sklearn.naive_bayes import MultinomialNB # sklearn이 제공하는 MultinomialNB 를 사용
NB_clf = MultinomialNB() # 분류기 선언

NB_clf.fit(X_train_cv, y_train) # train set을 이용하여 분류기(classifier)를 학습

print('Train set score: {:.3f}'.format(NB_clf.score(X_train_cv, y_train))) # train set에 대한 예측정확도를 확인
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_cv, y_test))) # test set에 대한 예측정확도를 확인

Train set score: 0.824
Test set score: 0.732


위 예시에서 MultinomialNB 클래스의 fit() 메서드는 주어진 특성 데이터와 라벨을 이용해 학습을 수행한다. X_train_cv는 카운트 벡터로 추출된 특성이고, y_train은 해당 문서에 대한 분류임을 기억하면 된다. Score() 메서드는 정확도를 반환하는데, 정확도를 계산하려면 예측을 위한 특성 값과 라벨이 모두 필요하므로 fit()과 마찬가지로 둘 다 인수로 전달한다. 위 결과를 보면 학습 데이터에 대한 정확도가 평가 데이터에 대한 정확도보다 높은데, 이는 머신러닝에서 자주 나타나는 현상이다. 하지만 두 값의 차이가 크다면 과적합으로 일반화 성능이 떨어진 것을 의심할 수 있다.

주어진 텍스트에 대해 분류를 예측하고 싶다면 predict() 메서드를 이용하면 된다. 예측헤는 라벨이 필요 없으므로 아래와 같이 특성 값만 인수로 주면 된다. 결과는 라벨과 동일하게 숫자로 반환되므로 실제 카테고리를 알고 싶다면 newsgroups_train.target_names를 이용해 아래와 같이 출력한다.

In [3]:
print('# First document and label in test data:', X_test[0], y_test[0])
print('# Second document and label in test data:', X_test[1], y_test[1])

pred = NB_clf.predict(X_test_cv[:2])

print('# Predicted labels:', pred)
print('# Predicted categories:', newsgroups_train.target_names[pred[0]], newsgroups_train.target_names[pred[1]])


# First document and label in test data: TRry the SKywatch project in  Arizona. 2
# Second document and label in test data: The Vatican library recently made a tour of the US.
 Can anyone help me in finding a FTP site where this collection is 
 available. 1
# Predicted labels: [2 1]
# Predicted categories: sci.space comp.graphics


평가를 반복해서 최종 모형을 도출하는 부분은 지금부터 다양한 시도를 함으로써 해볼 것이며, 그 외 실제 데이터에 대해 먼저 텍스트 데이터를 전처리하는 과정은 앞에서 전체 데이터에 대해 했던 과정을 동일하게 반복하면 된다. 주의할 점은 학습 데이터로 만든 CountVectorizer의 transform() 메서드를 그대로 사용해야 하며, 실제 데이터에 대해 새로 fit() 메서드를 적용해서는 안된다는 점이다. 모든 fit() 메서드는 학습 데이터에 대해서만 한 번 수행되고 평가 데이터나 실제 데이터에 대해서는 수행하지 않는다.

위에서 나온 결과를 개선할 방법은 수도 없이 많은데, 첬째로 max_features, min_df, max_df와 같은 CountVectorizer의 매개변수를 변경할 수 있다. 예를 들어 2,000개 대신 더 많은 수의 특성을 사용할 수 있다. 둘째로 나이브 베이즈의 매개변수를 조절할 수 있다. MultinomialNB는 모델의 복잡도를 조절하는 alpha 매개변수를 갖고 있다. 이 값을 늘리면 통계 데이터가 완만해지고 복잡도가 낮아진다. 셌재, 나이브 베이즈가 아닌 다른 머신러닝 알고리즘을 시도해 볼 수 있다. 넷째, CountVectorizer 대신 TfidfVectorizer를 사용해 볼 수 있다. 제안된 방법들을 앞으로 시도해볼 텐데, 우선 마지막 방법부터 시도해보도록 한다.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# CountVectorizer와 동일한 인수를 사용
tfidf = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train) # train set을 변환
X_test_tfidf = tfidf.transform(X_test) # test set을 변환

# tfidf train set을 이용해 분류기를 새로 학습
NB_clf.fit(X_train_tfidf, y_train)

# train set에 대한 예측 정확도를 확인
print(f'Train set score {NB_clf.score(X_train_tfidf, y_train):.3f}')
# test set에 대한 예측 정확도를 확인
print(f'Test set score {NB_clf.score(X_test_tfidf, y_test):.3f}')


Train set score 0.862
Test set score 0.741


위 결과를 보면 평가 데이터셍에 대한 정확도가 73.2%에서 74.1%로 올라갔다. 즉, TfidfVectorizer를 사용한 효과가 있다는 뜻이다.

앞에서 나이브 베이즈 분류기의 직관적인 원리에 대해 설명한 바와 같이, 분류에 대한 예측에는 특정 단어들이 기여한다. 네 가테고리별로 영향을 많이 미친 특성 혹은 단어를 알고 싶다면, 그 특성에 대해 학습된 계수를 보면 된다. 다음 코드에서는 카테고리별로 계수가 큰 10개의 특성들을 추출해 출력한다. 다른 알고리즘에 대해서도 사용할 수 있게 함수화한다.

In [5]:
import numpy as np

def top10_features(classifier, vectorizer, categories):
    feature_names = np.asarray(vectorizer.get_feature_names_out())
    for i, category in enumerate(categories):
        top10 = np.argsort(-classifier.coef_[i])[:10] # 역순으로 정렬하기 위해 계수에 음수를 취하여 정렬 후 앞에서부터 10개의 값을 반환
        print(f'{category}: {", ".join(feature_names[top10])}') # 카테고리와 영향이 큰 특성 10개를 출력

top10_features(NB_clf, tfidf, newsgroups_train.target_names)

alt.atheism: you, not, are, be, this, have, as, what, they, if
comp.graphics: you, on, graphics, this, have, any, can, or, with, thanks
sci.space: space, on, you, be, was, this, as, they, have, are
talk.religion.misc: you, not, he, are, as, this, be, god, was, they


/Users/sysanal/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `coef_` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


위 결과를 보면 흥미로운 부분들이 보이는데, atheism은 우리말로 무신론이란 뜻인데 상위 10개의 단어 중 이렇다 할 것이 별로 없다. 반면 'comp.graphics'는 'graphics'라는 결정적인 단어가 있고, 'sci.space'에는 'space', 'talk.religion.misc'에는 'god'이라는 단어가 있다. 물론 카테고리를 예측하는 데에 상위 10개만 쓰이는 것은 아니므로 다른 단어들의 영향도 봐야 알겠지만 you, are, this와 같은 단어들이 모든 카테고리에 동시에 있으면서 높은 영향을 미친다는 것은 의아하긴 하다. 이와 같이 나이브 베이즈에서의 coef_ 값은 결과를 해석하기 쉽지 않은데, 정확히 그런 이유인지는 알 수 없지만 sklearn 1.1 버전부터는 MultinomialNB 클래스에 대해 coef_가 더이상 지원되지 않는다. 따라서 coef_를 이용한 단어의 영향에 대한 분석은 로지스틱 회귀분석 계열에서 주로 사용하기를 권장한다.

In [6]:
% store tfidf
% store X_train_tfidf
% store X_test_tfidf

Stored 'tfidf' (TfidfVectorizer)
Stored 'X_train_tfidf' (csr_matrix)
Stored 'X_test_tfidf' (csr_matrix)
